In [1]:
#==Wide&Deep模型推荐==

# 1、出现背景
# CTR预估任务中，手工制做交叉特征有很好的“记忆性”，但耗精力，且“泛化性”不好（没出现过的特征组合权重系数为0）。
# Wide&Deep出现为了解决同时具备 “记忆性”Memorization、“泛化性”Generalization问题。
# 在Google Play store的场景中成功落地。

# 2、模型结构、原理
# wide部分和Deep部分。
# wide用于增强“记忆能力”：
# wide部分是一个广义的线性模型,输入的特征主要有两部分组成，一部分是原始的部分特征，另一部分是原始特征的交互特征(cross-product transformation)，
# 对于wide部分训练时候使用的优化器是带 正则的FTRL算法非常注重模型稀疏性质的，也就是说W&D模型采用L1 FTRL是想让Wide部分变得更加的稀疏，即Wide部分的大部分参数都为0，这就大大压缩了模型权重及特征向量的维度。
# ***Wide部分模型训练完之后留下来的特征都是非常重要的，那么模 型的“记忆能力”就可以理解为发现"直接的"，“暴力的”，“显然的”关联规则的能力。***


# Deep用于增强“泛化能力”：
# Deep部分是一个DNN模型,输入的特征主要分为两大类，一类是数值特征(可直接输入DNN)，一类是类别特征(需要经过Embedding之后才能输入到DNN中)，
# 我们知道DNN模型随着层数的增加，中间的特征就越抽象，也就提高了模型的泛化能力。
# Deep部分的DNN模型作者使用了深度学习常用的优化器AdaGrad，这也是为了使得模型可以得到更精确的解。

# Wide部分与Deep部分的结合:
# W&D模型是将两部分输出的结果结合起来联合训练，将deep和wide部分的输出重新使用一个逻辑回归模型做最终的预测，输出概率值。


# 3、操作流程参考

# 1. Retrieval检索：利用机器学习模型和一些人为定义的规则，来返回最匹配当前Query的一个小的items集合，这个集合就是最终的推荐列表的候选集。
# 2. Ranking排序：
# a. 收集更细致的用户特征，如：
# User features（年龄、性别、语言、民族等）
# Contextual features(上下文特征：设备，时间等)
# Impression features（展示特征：app age、app的历史统计信息等）
# b. 将特征分别传入Wide和Deep一起做训练。

# 训练时，根据最终的loss计算出gradient，反向传播到Wide和Deep两部分中，分别训练自己的参数（wide组件只需要填补deep组件的不足就行了，所以需要比较少的crossproduct feature transformations，而不是full-size wide Model）

# 训练方法是用mini-batch stochastic optimization。
# Wide组件是用FTRL（Follow-the-regularized-leader） + L1正则化学习。
# Deep组件是用AdaGrad来学习。

# c. 训练完之后推荐TopN

# 想要很好的使用wide&deep模型的话，还是要深入理解 业务，确定wide部分使用哪部分特征，deep部分使用哪些特征，以及wide部分的交叉特征应该如何去选择。



In [2]:
# 4、代码实现
# Wide侧记住的是历史数据中那些常见、高频的模式，是推荐系统中的“红海”。实际上，Wide侧没有发现新的模式，只是学习到这些模式之间的权重，做一些模式的筛选。正因为Wide侧不能发现新模式，因此我们需要根据人工经验、业务背景，将我们认为有价值的、显而易见的特征及特征组合，喂入Wide侧

# Deep侧就是DNN，通过embedding的方式将categorical/id特征映射成稠密向量，让DNN学习到这些特征之间的深层交叉，以增强扩展能力。

# 模型的实现与模型结构类似由deep和wide两部分组成，这两部分结构所需要的特征在上面已经说过了，针对当前数据集实现，我们在wide部分加入了所有可能的一阶特征，包括数值特征和类别特征的onehot都加进去了，其实也可以加入一些与wide&deep原论文中类似交叉特征。只要能够发现高频、常见模式的特征都可以放在wide侧，对于Deep部分，在本数据中放入了数值特征和类别特征的embedding特征，实际应用也需要根据需求进行选择。

In [3]:
# Wide&Deep 模型的wide部分及Deep部分的特征选择，应该根据实际的业务场景去确定哪些特征应该放在Wide部分，哪些特征应该放在Deep部分
def WideNDeep(linear_feature_columns, dnn_feature_columns):
    # 构建输入层，即所有特征对应的Input()层，这里使用字典的形式返回，方便后续构建模型
    dense_input_dict, sparse_input_dict = build_input_layers(linear_feature_columns + dnn_feature_columns)

    # 将linear部分的特征中sparse特征筛选出来，后面用来做1维的embedding
    linear_sparse_feature_columns = list(filter(lambda x: isinstance(x, SparseFeat), linear_feature_columns))

    # 构建模型的输入层，模型的输入层不能是字典的形式，应该将字典的形式转换成列表的形式
    # 注意：这里实际的输入与Input()层的对应，是通过模型输入时候的字典数据的key与对应name的Input层
    input_layers = list(dense_input_dict.values()) + list(sparse_input_dict.values())

    # Wide&Deep模型论文中Wide部分使用的特征比较简单，并且得到的特征非常的稀疏，所以使用了FTRL优化Wide部分（这里没有实现FTRL）
    # 但是是根据他们业务进行选择的，我们这里将所有可能用到的特征都输入到Wide部分，具体的细节可以根据需求进行修改
    linear_logits = get_linear_logits(dense_input_dict, sparse_input_dict, linear_sparse_feature_columns)
    
    # 构建维度为k的embedding层，这里使用字典的形式返回，方便后面搭建模型
    embedding_layers = build_embedding_layers(dnn_feature_columns, sparse_input_dict, is_linear=False)

    dnn_sparse_feature_columns = list(filter(lambda x: isinstance(x, SparseFeat), dnn_feature_columns))

    # 在Wide&Deep模型中，deep部分的输入是将dense特征和embedding特征拼在一起输入到dnn中
    dnn_logits = get_dnn_logits(dense_input_dict, sparse_input_dict, dnn_sparse_feature_columns, embedding_layers)
    
    # 将linear,dnn的logits相加作为最终的logits
    output_logits = Add()([linear_logits, dnn_logits])

    # 这里的激活函数使用sigmoid
    output_layer = Activation("sigmoid")(output_logits)

    model = Model(input_layers, output_layer)
    return model